In [24]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
import logging

In [25]:
log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')
def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))
def extract_hour(s):
    return s.apply(lambda x: int(x.split('-')[1][1:]))

base_path = 'data_set_0926'

In [26]:
# 加载邀请数据
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
logging.info("invite %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_1_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
logging.info("test %s", test.shape)

sub = test.copy()
sub_size = len(sub)

train['day'] = extract_day(train['dt'])
train['hour'] = extract_hour(train['dt'])
test['day'] = extract_day(test['dt'])
test['hour'] = extract_hour(test['dt'])

del train['dt'], test['dt']

[2019-12-11 16:08:14,437] INFO in <ipython-input-26-6060da740369>: invite (9489162, 4)
[2019-12-11 16:08:19,580] INFO in <ipython-input-26-6060da740369>: test (1141683, 3)


In [28]:
# 加载问题
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t')
ques.columns = ['qid', 'q_dt', 'title_t1', 'title_t2', 'desc_t1', 'desc_t2', 'topic']
del ques['title_t1'], ques['title_t2'], ques['desc_t1'], ques['desc_t2']
logging.info("ques %s", ques.shape)

ques['q_day'] = extract_day(ques['q_dt'])
ques['q_hour'] = extract_hour(ques['q_dt'])
del ques['q_dt']

[2019-12-11 16:19:14,389] INFO in <ipython-input-28-d1d53554f000>: ques (1829900, 3)


In [30]:
# 加载回答
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t')
ans.columns = ['aid', 'qid', 'uid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 'has_img',
               'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
               'reci_xxx', 'reci_no_help', 'reci_dis']
del ans['ans_t1'], ans['ans_t2']
logging.info("ans %s", ans.shape)

ans['a_day'] = extract_day(ans['ans_dt'])
ans['a_hour'] = extract_hour(ans['ans_dt'])
del ans['ans_dt']

ans = pd.merge(ans, ques, on='qid')
del ques

[2019-12-11 16:23:50,780] INFO in <ipython-input-30-0f04b531e5be>: ans (4513735, 18)


In [33]:
print(ans['a_day'].max())
print(train['day'].max())

3867
3867


In [7]:
# 回答距提问的天数
ans['diff_qa_days'] = ans['a_day'] - ans['q_day']

# 时间窗口划分
# train
# val
train_start = 3838
train_end = 3867

val_start = 3868
val_end = 3874

# 这里标示训练集起始
label_end = 3867
label_start = label_end - 6

train_label_feature_end = label_end - 7
train_label_feature_start = train_label_feature_end - 22

train_ans_feature_end = label_end - 7
train_ans_feature_start = train_ans_feature_end - 50

val_label_feature_end = val_start - 1
val_label_feature_start = val_label_feature_end - 22

val_ans_feature_end = val_start - 1
val_ans_feature_start = val_ans_feature_end - 50

train_label_feature = train[(train['day'] >= train_label_feature_start) & (train['day'] <= train_label_feature_end)]
logging.info("train_label_feature %s", train_label_feature.shape)

val_label_feature = train[(train['day'] >= val_label_feature_start) & (train['day'] <= val_label_feature_end)]
logging.info("val_label_feature %s", val_label_feature.shape)

train_label = train[(train['day'] > train_label_feature_end)]

logging.info("train feature start %s end %s, label start %s end %s", train_label_feature['day'].min(),
             train_label_feature['day'].max(), train_label['day'].min(), train_label['day'].max())

logging.info("test feature start %s end %s, label start %s end %s", val_label_feature['day'].min(),
             val_label_feature['day'].max(), test['day'].min(), test['day'].max())

# 确定ans的时间范围
# 3807~3874
train_ans_feature = ans[(ans['a_day'] >= train_ans_feature_start) & (ans['a_day'] <= train_ans_feature_end)]

val_ans_feature = ans[(ans['a_day'] >= val_ans_feature_start) & (ans['a_day'] <= val_ans_feature_end)]

logging.info("train ans feature %s, start %s end %s", train_ans_feature.shape, train_ans_feature['a_day'].min(),
             train_ans_feature['a_day'].max())

logging.info("val ans feature %s, start %s end %s", val_ans_feature.shape, val_ans_feature['a_day'].min(),
             val_ans_feature['a_day'].max())

fea_cols = ['is_good', 'is_rec', 'is_dest', 'has_img', 'has_video', 'word_count',
            'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
            'reci_xxx', 'reci_no_help', 'reci_dis', 'diff_qa_days']

[2019-12-08 12:20:39,555] INFO in <ipython-input-7-bcfa5b1b4c82>: train_label_feature (6895493, 5)
[2019-12-08 12:20:40,113] INFO in <ipython-input-7-bcfa5b1b4c82>: val_label_feature (7583553, 5)
[2019-12-08 12:20:40,381] INFO in <ipython-input-7-bcfa5b1b4c82>: train feature start 3838 end 3860, label start 3861 end 3867
[2019-12-08 12:20:40,407] INFO in <ipython-input-7-bcfa5b1b4c82>: test feature start 3845 end 3867, label start 3868 end 3874
[2019-12-08 12:20:44,046] INFO in <ipython-input-7-bcfa5b1b4c82>: train ans feature (3700178, 23), start 3810 end 3860
[2019-12-08 12:20:44,056] INFO in <ipython-input-7-bcfa5b1b4c82>: val ans feature (3992334, 23), start 3817 end 3867


In [14]:
print(train_label.columns)

Index(['qid', 'uid', 'label', 'day', 'hour', 'q_inv_mean', 'q_inv_sum',
       'q_inv_std', 'q_inv_count', 'u_inv_mean',
       ...
       'u_reci_dis_mean', 'q_reci_dis_sum', 'q_reci_dis_max',
       'q_reci_dis_mean', 'u_diff_qa_days_sum', 'u_diff_qa_days_max',
       'u_diff_qa_days_mean', 'q_diff_qa_days_sum', 'q_diff_qa_days_max',
       'q_diff_qa_days_mean'],
      dtype='object', length=105)


In [8]:
def extract_feature1(target, label_feature, ans_feature):
    # 问题特征
    t1 = label_feature.groupby('qid')['label'].agg(['mean', 'sum', 'std', 'count']).reset_index()
    t1.columns = ['qid', 'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count']
    target = pd.merge(target, t1, on='qid', how='left')

    # 用户特征
    t1 = label_feature.groupby('uid')['label'].agg(['mean', 'sum', 'std', 'count']).reset_index()
    t1.columns = ['uid', 'u_inv_mean', 'u_inv_sum', 'u_inv_std', 'u_inv_count']
    target = pd.merge(target, t1, on='uid', how='left')
    #
    # train_size = len(train)
    # data = pd.concat((train, test), sort=True)

    # 回答部分特征
    t1 = ans_feature.groupby('qid')['aid'].count().reset_index()
    t1.columns = ['qid', 'q_ans_count']
    target = pd.merge(target, t1, on='qid', how='left')

    t1 = ans_feature.groupby('uid')['aid'].count().reset_index()
    t1.columns = ['uid', 'u_ans_count']
    target = pd.merge(target, t1, on='uid', how='left')

    for col in fea_cols:
        t1 = ans_feature.groupby('uid')[col].agg(['sum', 'max', 'mean']).reset_index()
        t1.columns = ['uid', f'u_{col}_sum', f'u_{col}_max', f'u_{col}_mean']
        target = pd.merge(target, t1, on='uid', how='left')

        t1 = ans_feature.groupby('qid')[col].agg(['sum', 'max', 'mean']).reset_index()
        t1.columns = ['qid', f'q_{col}_sum', f'q_{col}_max', f'q_{col}_mean']
        target = pd.merge(target, t1, on='qid', how='left')
        logging.info("extract %s", col)
    return target

In [9]:
train_label = extract_feature1(train_label, train_label_feature, train_ans_feature)
test = extract_feature1(test, val_label_feature, val_ans_feature)

[2019-12-08 12:22:12,288] INFO in <ipython-input-8-0fafb00ac390>: extract is_good
[2019-12-08 12:22:26,776] INFO in <ipython-input-8-0fafb00ac390>: extract is_rec
[2019-12-08 12:22:39,626] INFO in <ipython-input-8-0fafb00ac390>: extract is_dest
[2019-12-08 12:22:52,316] INFO in <ipython-input-8-0fafb00ac390>: extract has_img
[2019-12-08 12:23:04,903] INFO in <ipython-input-8-0fafb00ac390>: extract has_video
[2019-12-08 12:23:17,551] INFO in <ipython-input-8-0fafb00ac390>: extract word_count
[2019-12-08 12:23:30,194] INFO in <ipython-input-8-0fafb00ac390>: extract reci_cheer
[2019-12-08 12:23:42,910] INFO in <ipython-input-8-0fafb00ac390>: extract reci_uncheer
[2019-12-08 12:23:55,914] INFO in <ipython-input-8-0fafb00ac390>: extract reci_comment
[2019-12-08 12:24:09,238] INFO in <ipython-input-8-0fafb00ac390>: extract reci_mark
[2019-12-08 12:24:22,657] INFO in <ipython-input-8-0fafb00ac390>: extract reci_tks
[2019-12-08 12:24:35,913] INFO in <ipython-input-8-0fafb00ac390>: extract reci

In [10]:
# 特征提取结束
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)
assert len(test) == sub_size

[2019-12-08 12:28:36,158] INFO in <ipython-input-10-464c91a59cf8>: train shape (2593669, 105), test shape (1141683, 104)


In [15]:
# 加载用户
user = pd.read_csv(f'{base_path}/member_info_0926.txt', header=None, sep='\t')
user.columns = ['uid', 'gender', 'creat_keyword', 'level', 'hot', 'reg_type', 'reg_plat', 'freq', 'uf_b1', 'uf_b2',
                'uf_b3', 'uf_b4', 'uf_b5', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5', 'score', 'follow_topic',
                'inter_topic']
del user['follow_topic'], user['inter_topic']
logging.info("user %s", user.shape)

unq = user.nunique()
logging.info("user unq %s", unq)

for x in unq[unq == 1].index:
    del user[x]
    logging.info('del unq==1 %s', x)

t = user.dtypes
cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
logging.info("user cat %s", cats)

for d in cats:
    lb = LabelEncoder()
    user[d] = lb.fit_transform(user[d])
    logging.info('encode %s', d)

q_lb = LabelEncoder()
q_lb.fit(list(train_label['qid'].astype(str).values) + list(test['qid'].astype(str).values))
train_label['qid_enc'] = q_lb.transform(train_label['qid'])
test['qid_enc'] = q_lb.transform(test['qid'])

u_lb = LabelEncoder()
u_lb.fit(user['uid'])
train_label['uid_enc'] = u_lb.transform(train_label['uid'])
test['uid_enc'] = u_lb.transform(test['uid'])

[2019-12-08 12:58:43,110] INFO in <ipython-input-15-342d21f23400>: user (1931654, 19)
[2019-12-08 12:58:46,408] INFO in <ipython-input-15-342d21f23400>: user unq uid              1931654
gender                 3
creat_keyword          1
level                  1
hot                    1
reg_type               1
reg_plat               1
freq                   5
uf_b1                  2
uf_b2                  2
uf_b3                  2
uf_b4                  2
uf_b5                  2
uf_c1               2561
uf_c2                291
uf_c3                428
uf_c4               1556
uf_c5                  2
score                732
dtype: int64
[2019-12-08 12:58:46,445] INFO in <ipython-input-15-342d21f23400>: del unq==1 creat_keyword
[2019-12-08 12:58:46,450] INFO in <ipython-input-15-342d21f23400>: del unq==1 level
[2019-12-08 12:58:46,452] INFO in <ipython-input-15-342d21f23400>: del unq==1 hot
[2019-12-08 12:58:46,637] INFO in <ipython-input-15-342d21f23400>: del unq==1 reg_type
[2019

In [20]:
user['score'].value_counts

<bound method IndexOpsMixin.value_counts of 0          764
1          671
2          454
3          588
4          361
          ... 
1931649    386
1931650    355
1931651    292
1931652    335
1931653    330
Name: score, Length: 1931654, dtype: int64>

In [17]:
# merge user
train_label = pd.merge(train_label, user, on='uid', how='left')
test = pd.merge(test, user, on='uid', how='left')
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)

data = pd.concat((train_label, test), axis=0, sort=True)
del test

# count编码
count_fea = ['uid_enc', 'qid_enc', 'gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
for feat in count_fea:
    col_name = '{}_count'.format(feat)
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data.loc[data[col_name] < 2, feat] = -1
    data[feat] += 1
    data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
    data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())
    # 

[2019-12-08 13:02:53,983] INFO in <ipython-input-17-82494adc454f>: train shape (2593669, 120), test shape (1141683, 119)


In [21]:
train_label.columns

Index(['qid', 'uid', 'label', 'day', 'hour', 'q_inv_mean', 'q_inv_sum',
       'q_inv_std', 'q_inv_count', 'u_inv_mean',
       ...
       'uf_b2', 'uf_b3', 'uf_b4', 'uf_b5', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4',
       'uf_c5', 'score'],
      dtype='object', length=120)

In [18]:
# 压缩数据
t = data.dtypes
for x in t[t == 'int64'].index:
    data[x] = data[x].astype('int32')

for x in t[t == 'float64'].index:
    data[x] = data[x].astype('float32')

data['wk'] = data['day'] % 7
feature_cols = [x for x in data.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]

In [22]:
# target编码
logging.info("feature size %s", len(feature_cols))

X_train_all = data.iloc[:len(train_label)][feature_cols]
y_train_all = data.iloc[:len(train_label)]['label']
test = data.iloc[len(train_label):]
del data
assert len(test) == sub_size
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

[2019-12-08 13:39:59,006] INFO in <ipython-input-22-870311050bbc>: feature size 126
[2019-12-08 13:39:59,986] INFO in <ipython-input-22-870311050bbc>: train shape (2593669, 120), test shape (1141683, 130)


In [23]:
X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
del train_label, X_train_all

model_lgb = LGBMClassifier(n_estimators=2000, n_jobs=6, objective='binary', seed=1000, silent=True,
                           device="gpu",gpu_platform_id = 0,gpu_device_id = 3,max_bin=63)

model_lgb.fit(X_train, y_train,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)

sub['label'] = model_lgb.predict_proba(test[feature_cols])[:, 1]

sub.to_csv('result7.txt', index=None, header=None, sep='\t')

[1]	valid_0's auc: 0.758611	valid_0's binary_logloss: 0.426598
Training until validation scores don't improve for 50 rounds
[2]	valid_0's auc: 0.77473	valid_0's binary_logloss: 0.416545
[3]	valid_0's auc: 0.780512	valid_0's binary_logloss: 0.408516
[4]	valid_0's auc: 0.783862	valid_0's binary_logloss: 0.401715
[5]	valid_0's auc: 0.78526	valid_0's binary_logloss: 0.396278
[6]	valid_0's auc: 0.788176	valid_0's binary_logloss: 0.391111
[7]	valid_0's auc: 0.790794	valid_0's binary_logloss: 0.386885
[8]	valid_0's auc: 0.792398	valid_0's binary_logloss: 0.383024
[9]	valid_0's auc: 0.79433	valid_0's binary_logloss: 0.379743
[10]	valid_0's auc: 0.796021	valid_0's binary_logloss: 0.376823
[11]	valid_0's auc: 0.797307	valid_0's binary_logloss: 0.374152
[12]	valid_0's auc: 0.798204	valid_0's binary_logloss: 0.371987
[13]	valid_0's auc: 0.799393	valid_0's binary_logloss: 0.37003
[14]	valid_0's auc: 0.800266	valid_0's binary_logloss: 0.368073
[15]	valid_0's auc: 0.801407	valid_0's binary_logloss: 0

[129]	valid_0's auc: 0.828282	valid_0's binary_logloss: 0.336499
[130]	valid_0's auc: 0.828319	valid_0's binary_logloss: 0.336465
[131]	valid_0's auc: 0.828364	valid_0's binary_logloss: 0.336428
[132]	valid_0's auc: 0.828407	valid_0's binary_logloss: 0.336388
[133]	valid_0's auc: 0.828411	valid_0's binary_logloss: 0.336381
[134]	valid_0's auc: 0.828439	valid_0's binary_logloss: 0.336356
[135]	valid_0's auc: 0.82852	valid_0's binary_logloss: 0.336264
[136]	valid_0's auc: 0.828587	valid_0's binary_logloss: 0.336209
[137]	valid_0's auc: 0.828657	valid_0's binary_logloss: 0.33615
[138]	valid_0's auc: 0.828687	valid_0's binary_logloss: 0.336127
[139]	valid_0's auc: 0.828746	valid_0's binary_logloss: 0.33608
[140]	valid_0's auc: 0.828755	valid_0's binary_logloss: 0.336073
[141]	valid_0's auc: 0.828777	valid_0's binary_logloss: 0.336056
[142]	valid_0's auc: 0.828838	valid_0's binary_logloss: 0.336013
[143]	valid_0's auc: 0.828909	valid_0's binary_logloss: 0.33596
[144]	valid_0's auc: 0.828945

[257]	valid_0's auc: 0.831456	valid_0's binary_logloss: 0.333842
[258]	valid_0's auc: 0.831459	valid_0's binary_logloss: 0.33384
[259]	valid_0's auc: 0.831456	valid_0's binary_logloss: 0.333841
[260]	valid_0's auc: 0.831465	valid_0's binary_logloss: 0.333836
[261]	valid_0's auc: 0.831485	valid_0's binary_logloss: 0.333822
[262]	valid_0's auc: 0.831516	valid_0's binary_logloss: 0.333778
[263]	valid_0's auc: 0.831524	valid_0's binary_logloss: 0.333771
[264]	valid_0's auc: 0.831526	valid_0's binary_logloss: 0.333768
[265]	valid_0's auc: 0.831526	valid_0's binary_logloss: 0.333768
[266]	valid_0's auc: 0.831539	valid_0's binary_logloss: 0.333757
[267]	valid_0's auc: 0.831548	valid_0's binary_logloss: 0.333749
[268]	valid_0's auc: 0.831551	valid_0's binary_logloss: 0.333745
[269]	valid_0's auc: 0.831549	valid_0's binary_logloss: 0.333747
[270]	valid_0's auc: 0.831551	valid_0's binary_logloss: 0.333745
[271]	valid_0's auc: 0.831566	valid_0's binary_logloss: 0.333733
[272]	valid_0's auc: 0.831

[385]	valid_0's auc: 0.832693	valid_0's binary_logloss: 0.332784
[386]	valid_0's auc: 0.832702	valid_0's binary_logloss: 0.332778
[387]	valid_0's auc: 0.832701	valid_0's binary_logloss: 0.332778
[388]	valid_0's auc: 0.832713	valid_0's binary_logloss: 0.332767
[389]	valid_0's auc: 0.832724	valid_0's binary_logloss: 0.332756
[390]	valid_0's auc: 0.83273	valid_0's binary_logloss: 0.33275
[391]	valid_0's auc: 0.832728	valid_0's binary_logloss: 0.332749
[392]	valid_0's auc: 0.832749	valid_0's binary_logloss: 0.332735
[393]	valid_0's auc: 0.832776	valid_0's binary_logloss: 0.332707
[394]	valid_0's auc: 0.832778	valid_0's binary_logloss: 0.332704
[395]	valid_0's auc: 0.832781	valid_0's binary_logloss: 0.332701
[396]	valid_0's auc: 0.832785	valid_0's binary_logloss: 0.332699
[397]	valid_0's auc: 0.8328	valid_0's binary_logloss: 0.332687
[398]	valid_0's auc: 0.832826	valid_0's binary_logloss: 0.332663
[399]	valid_0's auc: 0.83283	valid_0's binary_logloss: 0.33266
[400]	valid_0's auc: 0.832832	v

[512]	valid_0's auc: 0.83369	valid_0's binary_logloss: 0.331946
[513]	valid_0's auc: 0.833715	valid_0's binary_logloss: 0.331918
[514]	valid_0's auc: 0.833745	valid_0's binary_logloss: 0.331893
[515]	valid_0's auc: 0.833756	valid_0's binary_logloss: 0.331884
[516]	valid_0's auc: 0.833767	valid_0's binary_logloss: 0.331877
[517]	valid_0's auc: 0.833797	valid_0's binary_logloss: 0.331851
[518]	valid_0's auc: 0.833817	valid_0's binary_logloss: 0.331837
[519]	valid_0's auc: 0.833821	valid_0's binary_logloss: 0.331832
[520]	valid_0's auc: 0.833826	valid_0's binary_logloss: 0.331827
[521]	valid_0's auc: 0.833829	valid_0's binary_logloss: 0.331823
[522]	valid_0's auc: 0.83384	valid_0's binary_logloss: 0.331814
[523]	valid_0's auc: 0.833863	valid_0's binary_logloss: 0.331785
[524]	valid_0's auc: 0.833871	valid_0's binary_logloss: 0.33178
[525]	valid_0's auc: 0.833882	valid_0's binary_logloss: 0.331768
[526]	valid_0's auc: 0.833897	valid_0's binary_logloss: 0.331747
[527]	valid_0's auc: 0.83391

...

[1399]	valid_0's auc: 0.837323	valid_0's binary_logloss: 0.328882
[1400]	valid_0's auc: 0.837331	valid_0's binary_logloss: 0.328875
[1401]	valid_0's auc: 0.837343	valid_0's binary_logloss: 0.328867
[1402]	valid_0's auc: 0.837349	valid_0's binary_logloss: 0.328862
[1403]	valid_0's auc: 0.837353	valid_0's binary_logloss: 0.328858
[1404]	valid_0's auc: 0.837353	valid_0's binary_logloss: 0.328858
[1405]	valid_0's auc: 0.837354	valid_0's binary_logloss: 0.328857
[1406]	valid_0's auc: 0.83736	valid_0's binary_logloss: 0.328852
[1407]	valid_0's auc: 0.837367	valid_0's binary_logloss: 0.328848
[1408]	valid_0's auc: 0.837369	valid_0's binary_logloss: 0.328847
[1409]	valid_0's auc: 0.83737	valid_0's binary_logloss: 0.328845
[1410]	valid_0's auc: 0.837368	valid_0's binary_logloss: 0.328846
[1411]	valid_0's auc: 0.837364	valid_0's binary_logloss: 0.328849
[1412]	valid_0's auc: 0.837368	valid_0's binary_logloss: 0.328845
[1413]	valid_0's auc: 0.837366	valid_0's binary_logloss: 0.328847
[1414]	valid

[1525]	valid_0's auc: 0.837629	valid_0's binary_logloss: 0.328629
[1526]	valid_0's auc: 0.837626	valid_0's binary_logloss: 0.32863
[1527]	valid_0's auc: 0.83763	valid_0's binary_logloss: 0.328628
[1528]	valid_0's auc: 0.83763	valid_0's binary_logloss: 0.328627
[1529]	valid_0's auc: 0.837632	valid_0's binary_logloss: 0.328625
[1530]	valid_0's auc: 0.837629	valid_0's binary_logloss: 0.328628
[1531]	valid_0's auc: 0.837626	valid_0's binary_logloss: 0.328629
[1532]	valid_0's auc: 0.837624	valid_0's binary_logloss: 0.328632
[1533]	valid_0's auc: 0.837624	valid_0's binary_logloss: 0.328631
[1534]	valid_0's auc: 0.837625	valid_0's binary_logloss: 0.328631
[1535]	valid_0's auc: 0.837624	valid_0's binary_logloss: 0.328631
[1536]	valid_0's auc: 0.837623	valid_0's binary_logloss: 0.32863
[1537]	valid_0's auc: 0.837625	valid_0's binary_logloss: 0.328628
[1538]	valid_0's auc: 0.837629	valid_0's binary_logloss: 0.328625
[1539]	valid_0's auc: 0.837628	valid_0's binary_logloss: 0.328625
[1540]	valid_0

[1650]	valid_0's auc: 0.837879	valid_0's binary_logloss: 0.328428
[1651]	valid_0's auc: 0.837885	valid_0's binary_logloss: 0.328424
[1652]	valid_0's auc: 0.837894	valid_0's binary_logloss: 0.328416
[1653]	valid_0's auc: 0.837892	valid_0's binary_logloss: 0.328417
[1654]	valid_0's auc: 0.837891	valid_0's binary_logloss: 0.328419
[1655]	valid_0's auc: 0.837893	valid_0's binary_logloss: 0.328418
[1656]	valid_0's auc: 0.837897	valid_0's binary_logloss: 0.328415
[1657]	valid_0's auc: 0.8379	valid_0's binary_logloss: 0.328413
[1658]	valid_0's auc: 0.837907	valid_0's binary_logloss: 0.328408
[1659]	valid_0's auc: 0.837911	valid_0's binary_logloss: 0.328405
[1660]	valid_0's auc: 0.837917	valid_0's binary_logloss: 0.3284
[1661]	valid_0's auc: 0.837928	valid_0's binary_logloss: 0.328391
[1662]	valid_0's auc: 0.837928	valid_0's binary_logloss: 0.328391
[1663]	valid_0's auc: 0.83793	valid_0's binary_logloss: 0.328389
[1664]	valid_0's auc: 0.837946	valid_0's binary_logloss: 0.328377
[1665]	valid_0'

[1775]	valid_0's auc: 0.838136	valid_0's binary_logloss: 0.328218
[1776]	valid_0's auc: 0.838139	valid_0's binary_logloss: 0.328215
[1777]	valid_0's auc: 0.838145	valid_0's binary_logloss: 0.32821
[1778]	valid_0's auc: 0.838146	valid_0's binary_logloss: 0.32821
[1779]	valid_0's auc: 0.838148	valid_0's binary_logloss: 0.328209
[1780]	valid_0's auc: 0.838148	valid_0's binary_logloss: 0.328208
[1781]	valid_0's auc: 0.838149	valid_0's binary_logloss: 0.328208
[1782]	valid_0's auc: 0.838149	valid_0's binary_logloss: 0.328208
[1783]	valid_0's auc: 0.83815	valid_0's binary_logloss: 0.328207
[1784]	valid_0's auc: 0.838157	valid_0's binary_logloss: 0.328202
[1785]	valid_0's auc: 0.838163	valid_0's binary_logloss: 0.328196
[1786]	valid_0's auc: 0.838162	valid_0's binary_logloss: 0.328197
[1787]	valid_0's auc: 0.838165	valid_0's binary_logloss: 0.328195
[1788]	valid_0's auc: 0.838165	valid_0's binary_logloss: 0.328195
[1789]	valid_0's auc: 0.838167	valid_0's binary_logloss: 0.328194
[1790]	valid_

[1901]	valid_0's auc: 0.838335	valid_0's binary_logloss: 0.328048
[1902]	valid_0's auc: 0.838337	valid_0's binary_logloss: 0.328046
[1903]	valid_0's auc: 0.838341	valid_0's binary_logloss: 0.328043
[1904]	valid_0's auc: 0.838341	valid_0's binary_logloss: 0.328043
[1905]	valid_0's auc: 0.838339	valid_0's binary_logloss: 0.328044
[1906]	valid_0's auc: 0.838343	valid_0's binary_logloss: 0.328042
[1907]	valid_0's auc: 0.838341	valid_0's binary_logloss: 0.328045
[1908]	valid_0's auc: 0.838342	valid_0's binary_logloss: 0.328044
[1909]	valid_0's auc: 0.838343	valid_0's binary_logloss: 0.328043
[1910]	valid_0's auc: 0.838346	valid_0's binary_logloss: 0.328041
[1911]	valid_0's auc: 0.838346	valid_0's binary_logloss: 0.32804
[1912]	valid_0's auc: 0.838347	valid_0's binary_logloss: 0.32804
[1913]	valid_0's auc: 0.838353	valid_0's binary_logloss: 0.328035
[1914]	valid_0's auc: 0.838351	valid_0's binary_logloss: 0.328035
[1915]	valid_0's auc: 0.838356	valid_0's binary_logloss: 0.328031
[1916]	valid